In [2]:
import pandas as pd
import numpy as np
import requests
import zipfile
import io

# 1. VERİYİ İNDİRME VE YÜKLEME
# ---------------------------------------------------------
print("1. UCI Tüketim Verisi İndiriliyor... (İnternet hızına göre 1-2 dk sürebilir)")
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00235/household_power_consumption.zip"
response = requests.get(url)

print("2. Zip Dosyası Açılıyor ve Okunuyor... (Veri büyük, lütfen bekleyin)")
with zipfile.ZipFile(io.BytesIO(response.content)) as z:
    with z.open('household_power_consumption.txt') as f:
        # GÜNCELLEME: parse_dates ve infer_datetime_format uyarılarını kaldırdık.
        # Tarihleri daha sonra manuel birleştireceğiz.
        df_cons = pd.read_csv(f, sep=';', na_values=['?'], low_memory=False)

print("   -> Ham veri yüklendi. Tarih formatı düzenleniyor...")

# Tarih ve Saat sütunlarını birleştirip datetime formatına çeviriyoruz (Daha güvenli yöntem)
df_cons['dt'] = pd.to_datetime(df_cons['Date'] + ' ' + df_cons['Time'], dayfirst=True)
df_cons.set_index('dt', inplace=True)

# 2. VERİ ÖN İŞLEME (RESAMPLING)
# ---------------------------------------------------------
print("3. Dakikalık Veri Saatlik Formata Dönüştürülüyor...")

# Global_active_power (kW) -> Saatlik Tüketim
df_hourly = df_cons['Global_active_power'].resample('H').mean()

# Eksik verileri doldur (Forward Fill - yeni yöntem)
df_hourly = df_hourly.ffill()

# 3. ZAMAN KAYDIRMA (2020-2023 DÖNEMİNE TAŞIMA)
# ---------------------------------------------------------
print("4. Tarihler 2020-2023 Dönemine Eşitleniyor...")

target_start_date = pd.Timestamp('2020-01-01 00:00:00')
time_delta = target_start_date - df_hourly.index[0]
new_index = df_hourly.index + time_delta
df_hourly.index = new_index

# Sadece 2020-2023 arasını al
df_hourly = df_hourly.loc['2020-01-01':'2023-12-31']

# 4. ÖLÇEKLEME (SCALING) - KRİTİK EŞİK (5000 kWh/Yıl)
# ---------------------------------------------------------
print("5. Manisa 'Kritik Eşik' (Yıllık 5000 kWh) Profiline Ölçekleniyor...")

hedef_yillik_tuketim = 5000 # kWh
hedef_saatlik_ortalama_kw = hedef_yillik_tuketim / 365 / 24 
mevcut_ortalama_kw = df_hourly.mean() 
scaling_factor = hedef_saatlik_ortalama_kw / mevcut_ortalama_kw

# Watt cinsine çevir (x1000)
df_final_consumption_watts = df_hourly * scaling_factor * 1000 

df_tuketim = pd.DataFrame(df_final_consumption_watts)
df_tuketim.columns = ['Tuketim_W']

# 5. KAYDETME
# ---------------------------------------------------------
dosya_ismi = "temiz_tuketim_veri_5000kWh.csv"
df_tuketim.to_csv(dosya_ismi)

print("-" * 40)
print(f"✅ İŞLEM BAŞARILI!")
print(f"Dosya Kaydedildi: {dosya_ismi}")
print(f"Veri Aralığı: {df_tuketim.index.min()} - {df_tuketim.index.max()}")
print("-" * 40)
print(df_tuketim.head())

1. UCI Tüketim Verisi İndiriliyor... (İnternet hızına göre 1-2 dk sürebilir)
2. Zip Dosyası Açılıyor ve Okunuyor... (Veri büyük, lütfen bekleyin)
   -> Ham veri yüklendi. Tarih formatı düzenleniyor...
3. Dakikalık Veri Saatlik Formata Dönüştürülüyor...
4. Tarihler 2020-2023 Dönemine Eşitleniyor...
5. Manisa 'Kritik Eşik' (Yıllık 5000 kWh) Profiline Ölçekleniyor...


C:\Users\Monster\AppData\Local\Temp\ipykernel_19432\2399517386.py:31: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_hourly = df_cons['Global_active_power'].resample('H').mean()


----------------------------------------
✅ İŞLEM BAŞARILI!
Dosya Kaydedildi: temiz_tuketim_veri_5000kWh.csv
Veri Aralığı: 2020-01-01 00:00:00 - 2023-12-12 04:00:00
----------------------------------------
                       Tuketim_W
dt                              
2020-01-01 00:00:00  2213.278669
2020-01-01 01:00:00  1903.689866
2020-01-01 02:00:00  1782.112697
2020-01-01 03:00:00  1713.104245
2020-01-01 04:00:00  1601.939491
